In [31]:
!pip install -q -U google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [37]:
# Importar bibliotecas

import os
from google.colab import userdata # Para usar no Colab. Depois usaremos variáveis de ambiente.
from google import genai
from google.genai import types  # Importante para usar GenerateContentConfig
from google.adk.tools import google_search # Ferramenta de busca
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
import textwrap # Para formatar texto
from IPython.display import display, Markdown # Para exibir formatado no Colab
import warnings

warnings.filterwarnings("ignore")

In [38]:
# Configurar API Key
# No Colab, lê da variável secreta
# Fora do Colab, configure a variável de ambiente GOOGLE_API_KEY no seu sistema

os.environ ['GOOGLE_API_KEY'] = userdata.get ('GEMINI_API_KEY')

In [21]:
# Definição de modelo
model_id = "gemini-2.0-flash"
print(f"Modelo selecionado: {model_id}")

Modelo selecionado: gemini-2.0-flash


In [39]:
# Funções Auxiliares para Agentes (se usou)
# Adaptação da função call_agent para ambiente Colab (usa display do IPython)
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="colab_user", session_id="colab_session")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    print(f"DEBUG: Acionando agente '{agent.name}'...")
    for event in runner.run(user_id="colab_user", session_id="colab_session", new_message=content):
        if event.is_final_response():
            print(f"DEBUG: Agente '{agent.name}' retornou resposta final.")
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

# Função auxiliar para exibir Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
# Definir a Configuração do Chat (incluindo a System Instruction crucial)
# Usamos types.GenerateContentConfig para definir as instruções do sistema

chat_config =types.GenerateContentConfig(
    system_instruction= """
    Você é o Assistente VivaCare, um chatbot amigável e informativo focado em fornecer informações gerais e acessíveis sobre transplante de órgãos (como fígado, rim, coração, pulmão, etc.) e cuidados pós-transplante.
    Seu objetivo principal é auxiliar pacientes, cuidadores ou pessoas interessadas com dúvidas básicas sobre o processo, recuperação, rotina e bem-estar pós-transplante.
    Sua linguagem deve ser clara, simples, empática e fácil de entender para quem não tem conhecimento médico profundo.
    Você deve manter um tom de suporte e encorajamento.

    É ABSOLUTAMENTE ESSENCIAL que, em **todas as suas respostas** que abordem temas de saúde, tratamento, medicação, sintomas, dieta, exercícios, ou qualquer conselho médico/de cuidado, você inclua a seguinte **RESSALVA DE FORMA CLARA E VISÍVEL**:

    ---
    **Importante:** Eu sou uma inteligência artificial e não substituo um profissional de saúde. As informações que forneço são para fins educacionais e de apoio geral e **não constituem aconselhamento médico**. Sempre consulte seu médico, equipe de transplante, nutricionista ou outro profissional de saúde qualificado para obter diagnóstico, tratamento, conselhos e orientações personalizadas para sua condição específica. Não tome decisões sobre sua saúde ou mude seu tratamento com base apenas nas informações que eu forneço.
    ---

    Responda apenas perguntas diretamente relacionadas a transplante de órgãos e cuidados pós-transplante. Se a pergunta for completamente fora deste tema, diga educadamente que sua especialidade é transplante e você não pode ajudar com aquele tópico.

    Não invente informações ou procedimentos médicos. Se você não tem certeza sobre algo, diga que não tem a informação ou que o usuário deve consultar um profissional.

    Esteja pronto para receber perguntas em português do Brasil.
    """
)

In [33]:
# Criar a sessão de chat com a configuração definida
# A sessão de chat mantém o histórico da conversa

try:
  model = genai.generativeModel(
      model_name = model_id,
      generation_config = chat_config,
      tools = [google_search]
  )
  chat_session = model.start_chat()
  print("Assistente VivaCare inicializado. Como posso te ajudar hoje?")
  print("(Digite 'fim' a qualquer momento para encerrar a conversa)")

except Exception as e:
  print(f"Erro ao iniciar sistema: {e}")
  chat_session = None

Erro ao iniciar sistema: module 'google.genai' has no attribute 'generativeModel'


In [35]:
# Loop de interação básica

if chat_session:
  prompt = input("Você: ")

  while prompt.lower() != "fim":
    try:

      # --- Lógica de Orquestração ---
      if "buscar recursos sobre" in prompt.lower():
        termo = prompt.lower().replace("buscar recurso sobre", "").strip()
        print("\nAcionando agentes para buscar e formatar recursos...")
        resultados_brutos = agentes_buscador_recursos(termo)
        resposta_final = agente_formatador_recursos(resultados_brutos)
        print("\nAssistente (via agentes):", resposta_final)
      elif "alguma outra tarefa específica" in prompt.lower():
        pass

        # Chamar outra cadeia de agentes
      else:
        response = chat_session.send_message(prompt)
        print("\nAssistente (chat básico):", response.text)

    except Exception as e:
      print(f"\nOcorreu um erro ao processar sua mensagem: {e}")
      print("Por favor, tente novamente.")

  print("-" * 30)  # Linha separadora para clareza
  prompt = input("Você: ")

print("\nObrigado por usar o Assistente VivaCare! Cuide-se!")


Obrigado por usar o Assistente VivaCare! Cuide-se!
